In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e17/sample_submission.csv
/kaggle/input/playground-series-s3e17/train.csv
/kaggle/input/playground-series-s3e17/test.csv


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
data_train=pd.read_csv("/kaggle/input/playground-series-s3e17/train.csv")
data_test=pd.read_csv("/kaggle/input/playground-series-s3e17/test.csv")

data_test['Machine failure']=np.nan
data_train['data']='train'
data_test['data']='test'
data_test=data_test[data_train.columns]

In [4]:
original_columns = [
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]'
]

new_columns = {col: col.split(' [')[0] for col in original_columns}
data_train = data_train.rename(columns=new_columns)

data_test = data_test.rename(columns=new_columns)
print(data_test.columns)

Index(['id', 'Product ID', 'Type', 'Air temperature', 'Process temperature',
       'Rotational speed', 'Torque', 'Tool wear', 'Machine failure', 'TWF',
       'HDF', 'PWF', 'OSF', 'RNF', 'data'],
      dtype='object')


In [5]:
data_all=pd.concat([data_train,data_test],axis=0)

In [6]:
data_all.head()

,id,Product ID,Type,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,Machine failure,TWF,HDF,PWF,OSF,RNF,data
0,0,L50096,L,300.6,309.6,1596,36.1,140,0.0,0,0,0,0,0,train
1,1,M20343,M,302.6,312.1,1759,29.1,200,0.0,0,0,0,0,0,train
2,2,L49454,L,299.3,308.5,1805,26.5,25,0.0,0,0,0,0,0,train
3,3,L53355,L,301.0,310.9,1524,44.3,197,0.0,0,0,0,0,0,train
4,4,M24050,M,298.0,309.0,1641,35.4,34,0.0,0,0,0,0,0,train


In [7]:
data_all.tail()

,id,Product ID,Type,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,Machine failure,TWF,HDF,PWF,OSF,RNF,data
90949,227378,L51130,L,302.3,311.4,1484,40.4,15,NaN,0,0,0,0,0,test
90950,227379,L47783,L,297.9,309.8,1542,33.8,31,NaN,0,0,0,0,0,test
90951,227380,L48097,L,295.6,306.2,1501,41.4,187,NaN,0,0,0,0,0,test
90952,227381,L48969,L,298.1,307.8,1534,40.3,69,NaN,0,0,0,0,0,test
90953,227382,L52525,L,303.5,312.8,1534,36.1,92,NaN,0,0,0,0,0,test


In [8]:
data_train.isna().sum()

id                     0
Product ID             0
Type                   0
Air temperature        0
Process temperature    0
Rotational speed       0
Torque                 0
Tool wear              0
Machine failure        0
TWF                    0
HDF                    0
PWF                    0
OSF                    0
RNF                    0
data                   0
dtype: int64

In [9]:
data_all['Machine failure'].value_counts()

Machine failure
0.0    134281
1.0      2148
Name: count, dtype: int64

In [10]:
134281/2148

62.51443202979516

In [11]:
list(zip(data_all.columns,data_all.dtypes,data_all.nunique()))

[('id', dtype('int64'), 227383),
 ('Product ID', dtype('O'), 9995),
 ('Type', dtype('O'), 3),
 ('Air temperature', dtype('float64'), 95),
 ('Process temperature', dtype('float64'), 84),
 ('Rotational speed', dtype('int64'), 974),
 ('Torque', dtype('float64'), 628),
 ('Tool wear', dtype('int64'), 246),
 ('Machine failure', dtype('float64'), 2),
 ('TWF', dtype('int64'), 2),
 ('HDF', dtype('int64'), 2),
 ('PWF', dtype('int64'), 2),
 ('OSF', dtype('int64'), 2),
 ('RNF', dtype('int64'), 2),
 ('data', dtype('O'), 2)]

In [12]:
types=pd.get_dummies(data_all['Type'], drop_first=False)


In [13]:
types.head()

,H,L,M
0,False,True,False
1,False,False,True
2,False,True,False
3,False,True,False
4,False,False,True


In [14]:
mapping_boolean = {True: 1, False: 0}
types['H'] = types['H'].map(mapping_boolean)
types['L'] = types['L'].map(mapping_boolean)
types['M'] = types['M'].map(mapping_boolean)

In [15]:
types.head()

,H,L,M
0,0,1,0
1,0,0,1
2,0,1,0
3,0,1,0
4,0,0,1


In [16]:
data_all=pd.concat([data_all, types], axis=1)


In [17]:
data_all.head()

,id,Product ID,Type,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,Machine failure,TWF,HDF,PWF,OSF,RNF,data,H,L,M
0,0,L50096,L,300.6,309.6,1596,36.1,140,0.0,0,0,0,0,0,train,0,1,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0.0,0,0,0,0,0,train,0,0,1
2,2,L49454,L,299.3,308.5,1805,26.5,25,0.0,0,0,0,0,0,train,0,1,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0.0,0,0,0,0,0,train,0,1,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0.0,0,0,0,0,0,train,0,0,1


In [18]:
data_all.drop(['id', 'Product ID', 'Type'], axis=1, inplace=True)


In [19]:
data_all.head()

,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,Machine failure,TWF,HDF,PWF,OSF,RNF,data,H,L,M
0,300.6,309.6,1596,36.1,140,0.0,0,0,0,0,0,train,0,1,0
1,302.6,312.1,1759,29.1,200,0.0,0,0,0,0,0,train,0,0,1
2,299.3,308.5,1805,26.5,25,0.0,0,0,0,0,0,train,0,1,0
3,301.0,310.9,1524,44.3,197,0.0,0,0,0,0,0,train,0,1,0
4,298.0,309.0,1641,35.4,34,0.0,0,0,0,0,0,train,0,0,1


In [20]:
data_train=data_all[data_all['data']=='train']
del data_train['data']
data_test=data_all[data_all['data']=='test']
data_test.drop(['Machine failure','data'],axis=1,inplace=True)

/tmp/ipykernel_18/1697743391.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.drop(['Machine failure','data'],axis=1,inplace=True)


In [21]:
data_train.head()

,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,Machine failure,TWF,HDF,PWF,OSF,RNF,H,L,M
0,300.6,309.6,1596,36.1,140,0.0,0,0,0,0,0,0,1,0
1,302.6,312.1,1759,29.1,200,0.0,0,0,0,0,0,0,0,1
2,299.3,308.5,1805,26.5,25,0.0,0,0,0,0,0,0,1,0
3,301.0,310.9,1524,44.3,197,0.0,0,0,0,0,0,0,1,0
4,298.0,309.0,1641,35.4,34,0.0,0,0,0,0,0,0,0,1


In [22]:
data_test.head()

,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,TWF,HDF,PWF,OSF,RNF,H,L,M
0,302.3,311.5,1499,38.0,60,0,0,0,0,0,0,1,0
1,301.7,311.0,1713,28.8,17,0,0,0,0,0,0,1,0
2,301.3,310.4,1525,37.7,96,0,0,0,0,0,0,1,0
3,300.1,309.6,1479,47.6,5,0,0,0,0,0,0,0,1
4,303.4,312.3,1515,41.3,114,0,0,0,0,0,0,0,1


In [23]:
from sklearn.model_selection import train_test_split

X = data_train.drop(columns="Machine failure")
y = data_train["Machine failure"]


X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state=0)

In [24]:
X_train.shape, X_val.shape

((102321, 13), (34108, 13))

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val= scaler.transform(X_val)

In [26]:
xgb_model = xgb.XGBClassifier()
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')  # OvO for multi-class
print(f'XGBoost Cross-Validation ROC-AUC: {xgb_scores.mean():.4f}')

# Random Forest model with ROC-AUC
rf_model = RandomForestClassifier()
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'Random Forest Cross-Validation ROC-AUC: {rf_scores.mean():.4f}')

# k-Nearest Neighbors (KNN) model with ROC-AUC
knn_model = KNeighborsClassifier()
knn_scores = cross_val_score(knn_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'KNN Cross-Validation ROC-AUC: {knn_scores.mean():.4f}')

# Logistic Regression model with ROC-AUC
logreg_model = LogisticRegression(class_weight = 'balanced')
logreg_scores = cross_val_score(logreg_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'Logistic Regression Cross-Validation ROC-AUC: {logreg_scores.mean():.4f}')

dt_model = DecisionTreeClassifier()
dt_scores = cross_val_score(dt_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'Decision Tree Cross-Validation ROC-AUC: {dt_scores.mean():.4f}')

# Support Vector Machine (SVM) model with ROC-AUC
svm_model = SVC(probability=True)
svm_scores = cross_val_score(svm_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'SVM Cross-Validation ROC-AUC: {svm_scores.mean():.4f}')

XGBoost Cross-Validation ROC-AUC: 0.9527
Random Forest Cross-Validation ROC-AUC: 0.9404
KNN Cross-Validation ROC-AUC: 0.8889


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistic Regression Cross-Validation ROC-AUC: 0.9328
Decision Tree Cross-Validation ROC-AUC: 0.8815
SVM Cross-Validation ROC-AUC: 0.9151


In [27]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_val)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
y_pred_proba = xgb_model.predict_proba(X_val)[:, 1]
y_pred_proba

array([0.00189464, 0.00475762, 0.00017893, ..., 0.00080381, 0.00013707,
       0.00014059], dtype=float32)